# Classifier Training

Make image size and crop size the same.

In [1]:
! rsync -a /kaggle/input/mmdetection-v280/mmdetection /
! pip install /kaggle/input/mmdetection-v280/src/mmpycocotools-12.0.3/mmpycocotools-12.0.3/
! pip install /kaggle/input/hpapytorchzoo/pytorch_zoo-master/
! pip install /kaggle/input/hpacellsegmentation/HPA-Cell-Segmentation/
! pip install /kaggle/input/iterative-stratification/iterative-stratification-master/

! cp -r /kaggle/input/kgl-humanprotein-data/kgl_humanprotein_data /
! cp -r /kaggle/input/humanpro/kgl_humanprotein /

import sys
sys.path.append('/kgl_humanprotein/')

Processing /kaggle/input/mmdetection-v280/src/mmpycocotools-12.0.3/mmpycocotools-12.0.3
  Created wheel for mmpycocotools: filename=mmpycocotools-12.0.3-cp37-cp37m-linux_x86_64.whl size=272915 sha256=21c802f514d5c4968fc959af7f5dda7204a64111a6d78ae9fdfb0787dc48bf21
  Stored in directory: /root/.cache/pip/wheels/80/e0/da/3288fdf3965b5c9090f368462db9d28be2c82013f51821090a
Successfully built mmpycocotools
Processing /kaggle/input/hpapytorchzoo/pytorch_zoo-master
  Created wheel for pytorch-zoo: filename=pytorch_zoo-0.0.0-py3-none-any.whl size=30139 sha256=569f1c4d21b90403367fe780a0f7a4235526fecd6f1ca0b195293135627d6f19
  Stored in directory: /root/.cache/pip/wheels/7f/18/21/aff5a8914e22461b2b025a9629c2b70464c36183caaf12bc09
Successfully built pytorch-zoo
Processing /kaggle/input/hpacellsegmentation/HPA-Cell-Segmentation
  Created wheel for hpacellseg: filename=hpacellseg-0.1.8-py3-none-any.whl size=14815 sha256=46f6c5187b355c1659f49219f0455a8daa980d34330451a50e3bdd2419e1a1d1
  Stored in di

In [2]:
import os
import time
from pathlib import Path
import shutil
import zipfile
import functools
import multiprocessing
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import KFold,StratifiedKFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import torch
from torch.backends import cudnn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn import DataParallel
import matplotlib.pyplot as plt
from tqdm import tqdm

from kgl_humanprotein.utils.common_util import *
from kgl_humanprotein.config.config import *
from kgl_humanprotein.data_process import *
from kgl_humanprotein.datasets.tool import image_to_tensor
from kgl_humanprotein.networks.imageclsnet import init_network
from kgl_humanprotein.layers.loss import *
from kgl_humanprotein.layers.scheduler import *
from kgl_humanprotein.utils.augment_util import train_multi_augment2
from kgl_humanprotein.utils.log_util import Logger
from kgl_humanprotein.run.train import *

run on 52b0d1eadd16


In [3]:
%cd /kaggle

/kaggle


## Write out 6 image-level samples from each subset for testing on laptop

In [4]:
# import shutil

In [5]:
# def generate_testing_samples(isubset, n_images=6, sz_img=384, dir_out=None):
#     dir_subset = Path(f'/kaggle/input/humanpro-train-cells-subset{isubset}')
#     dir_subset = dir_subset / f'humanpro_train_cells_subset{isubset}/train'
    
#     df = pd.read_feather(dir_subset / 'train.feather')
#     imgids = df['Id'].apply(lambda o: o.split('_')[0])
#     sample_imgids = np.random.choice(imgids.unique(), n_images)
#     df_sample = df[imgids.isin(sample_imgids)].reset_index(drop=True)
    
#     if dir_out is not None:
#         dir_subset_out = dir_out/'train'
#         dir_subset_out.mkdir(exist_ok=True, parents=True)
#         df_sample.to_feather(dir_subset_out/'train.feather')
        
#         dir_img = dir_subset/f'images_{sz_img}'
#         dir_img_out = dir_subset_out/f'images_{sz_img}'
#         dir_img_out.mkdir(exist_ok=True, parents=True)
#         for _, row in df_sample.iterrows():
#             srcs = list(dir_img.glob(f"{row['Id']}*.png"))
#             for src in srcs:
#                 shutil.copy(src, dir_img_out/src.name)
                
#     return df_sample

In [6]:
# n_subsets = 5

# for isubset in range(n_subsets):
#     print(f'\rProcessing subset {isubset}...', end='', flush=True)
#     dir_out = Path(f'/kaggle/working/humanpro_train_cells_subset{isubset}')
#     generate_testing_samples(isubset, dir_out=dir_out)

In [7]:
# ! zip -qr humanpro_train_cells_subset0.zip humanpro_train_cells_subset0/
# ! zip -qr humanpro_train_cells_subset1.zip humanpro_train_cells_subset1/
# ! zip -qr humanpro_train_cells_subset2.zip humanpro_train_cells_subset2/
# ! zip -qr humanpro_train_cells_subset3.zip humanpro_train_cells_subset3/
# ! zip -qr humanpro_train_cells_subset4.zip humanpro_train_cells_subset4/

## Combine subsets' meta data

In [8]:
dir_data = Path('/kaggle/input')
dir_mdata = Path('/kaggle/mdata')
n_subsets = 5
# sz_img = 384

In [9]:
%%time
df_cells = combine_subsets_metadata(dir_data, n_subsets)

Processing subset 4...CPU times: user 1.83 s, sys: 1.09 s, total: 2.92 s
Wall time: 8.27 s


In [10]:
dir_mdata_raw = dir_mdata/'raw'
dir_mdata_raw.mkdir(exist_ok=True, parents=True)

df_cells.to_feather(dir_mdata_raw/'train.feather')

In [11]:
del df_cells

## Filter samples

In [12]:
# Keep single labels
df_cells = pd.read_feather(dir_mdata_raw/'train.feather')
df_cells = (df_cells[df_cells['Target'].apply(lambda o: len(o.split('|'))==1)]
            .reset_index(drop=True))

In [13]:
# Limit number of samples per label

def cap_number_per_label(df_cells, cap=10_000, idx_start=0):
    df_cells_cap = pd.DataFrame()
    for label in df_cells.Target.unique():
        df = df_cells[df_cells.Target==label]
        if len(df) > cap:
            df = df.iloc[idx_start:idx_start + cap]
        df_cells_cap = df_cells_cap.append(df, ignore_index=True)
    return df_cells_cap

In [14]:
df_cells = cap_number_per_label(df_cells, cap=10_000, idx_start=0) 

In [15]:
df_cells.Target.value_counts()

14    10000
12    10000
16    10000
7     10000
4     10000
13    10000
8     10000
2     10000
5     10000
0     10000
3     10000
6     10000
10     7789
17     5619
9      5322
1      4845
15     2275
18      952
11       10
Name: Target, dtype: int64

In [16]:
df_cells.to_feather(dir_mdata_raw/'train.feather')

## One-hot encode labels

In [17]:
%%time
generate_meta(dir_mdata, 'train.feather')

CPU times: user 59.8 s, sys: 662 ms, total: 1min
Wall time: 1min


## Split generation

In [18]:
%%time
train_meta = pd.read_feather(dir_mdata/'meta'/'train_meta.feather')
create_random_split(dir_mdata, train_meta, n_splits=5, alias='random')
del train_meta

Nucleoplasm 8000 2000
Nuclear membrane 3876 969
Nucleoli 8000 2000
Nucleoli fibrillar center 8000 2000
Nuclear speckles 8000 2000
Nuclear bodies 8000 2000
Endoplasmic reticulum 8000 2000
Golgi apparatus 8000 2000
Intermediate filaments 8000 2000
Actin filaments 4258 1064
Microtubules 6232 1557
Mitotic spindle 8 2
Centrosome 8000 2000
Plasma membrane 8000 2000
Mitochondria 8000 2000
Aggresome 1820 455
Cytosol 8000 2000
Vesicles and punctate cytosolic patterns 4495 1124
Negative 761 191
create split file: /kaggle/mdata/split/random_folds5/random_train_cv0.feather, shape: (117450, 26)
create split file: /kaggle/mdata/split/random_folds5/random_valid_cv0.feather, shape: (29362, 26)
create split file: /kaggle/mdata/split/random_folds5/random_train_cv1.feather, shape: (117450, 26)
create split file: /kaggle/mdata/split/random_folds5/random_valid_cv1.feather, shape: (29362, 26)
create split file: /kaggle/mdata/split/random_folds5/random_train_cv2.feather, shape: (117449, 26)
create split file

## Training

In [19]:
model_multicell = (
    '../../kgl_humanprotein_data/result/models/'
    'external_crop512_focal_slov_hardlog_class_densenet121_dropout_i768_aug2_5folds/'
    'fold0/final.pth')
gpu_id = '0' # '0,1,2,3'
arch = 'class_densenet121_dropout'
num_classes = len(LABEL_NAME_LIST)
scheduler = 'Adam55'
epochs = 10 #55
resume = Path('/kaggle/input/humanpro-classifier/results/models/'
              'external_crop384_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0/final.pth')
sz_img = 384
crop_size = 384
batch_size = 32
split_name = 'random_folds5'
fold = 0
workers = 3
pin_memory = True

dir_results = Path('results')
dir_results.mkdir(exist_ok=True, parents=True)

out_dir = Path(f'external_crop{crop_size}_focal_slov_hardlog_class_densenet121_dropout_i{sz_img}_aug2_5folds')

In [20]:
main_training(dir_data, dir_mdata, dir_results, out_dir, 
              split_name=split_name, fold=fold,
              arch=arch, model_multicell=model_multicell, scheduler=scheduler,
              epochs=epochs, resume=resume,
              img_size=sz_img, crop_size=crop_size, batch_size=batch_size, 
              gpu_id=gpu_id, workers=workers, pin_memory=pin_memory)

>> Creating directory if it does not exist:
>> 'results/models/external_crop384_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0'
>> Using pre-trained model.
>> Loading multi-cell model.
>> Loading checkpoint:
>> '/kaggle/input/humanpro-classifier/results/models/external_crop384_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0/final.pth'
>> Loading checkpoint:
>> '/kaggle/input/humanpro-classifier/results/models/external_crop384_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0/final_optim.pth'
>>>> loaded checkpoint:
>>>> '/kaggle/input/humanpro-classifier/results/models/external_crop384_focal_slov_hardlog_class_densenet121_dropout_i384_aug2_5folds/fold0/final.pth' (epoch 3)
** start training here! **

epoch    iter      rate     |  train_loss/acc  |    valid_loss/acc/focal/kaggle     |best_epoch/best_focal|  min 
-----------------------------------------------------------------------------------------------------------------
  4

In [21]:
! cp -r results/ /kaggle/working/.